In [ ]:
import h5py
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
%matplotlib notebook

# Data access

In [ ]:
runnr = 100
base_path = '/Users/benedikt/data/LCLS/amol3416/pbcv/'
filename_ol1 = base_path + 'amol3416_r%04d_ol1.h5' %runnr
filename_ol3 = base_path + 'amol3416_r%04d_ol3.h5' %runnr
filename_sizing = base_path + 'sizing/amol3416_r%04d_sizing.h5' %runnr

# Hit-finding

In [ ]:
# Loading hitscore for output level 1 (all events)
with h5py.File(filename_ol1, 'r') as f:
    timestamp_all = f['entry_1/event/timestamp'][:]
    hitscore_all  = f['entry_1/result_1/hitscore_litpixel'][:]

# Loading hitscore for output level 3 (all hits)
with h5py.File(filename_ol3, 'r') as f:
    timestamp_hits = f['entry_1/event/timestamp'][:]
    hitscore_hits = f['entry_1/result_1/hitscore_litpixel'][:]
    hitscore_threshold = f['entry_1/result_1/hitscore_litpixel_threshold'][0]

# Time limits
time_limits = {}
time_limits['0098'] = [3000,-1000]
time_limits['0100'] = [0,13000]
tmin = timestamp_all[time_limits['%04d' %runnr][0]]
tmax = timestamp_all[time_limits['%04d' %runnr][1]]

# Valid (within time limits)
valid_all  = (timestamp_all > tmin) & (timestamp_all < tmax)
valid_hits = (timestamp_hits > tmin) & (timestamp_hits < tmax)

# Calculate overall hitrate
hitrate = float(valid_hits.sum()) / valid_all.sum() * 100
print "The overall hitrate for run %d is %.1f%% (%d hits)" %(runnr, hitrate, valid_hits.sum())
    
# Plotting hitscore and threshold
fig = plt.figure()
ax  = fig.add_subplot(111)
ax.set_title('Hitfinding (run %d) - %d hits' %(runnr, valid_hits.sum()))
ax.scatter(timestamp_all,  hitscore_all,  color='k', s=1, marker='.')
ax.scatter(timestamp_hits, hitscore_hits, color='r', s=1, marker='.')
ax.axvspan(tmin, tmax, alpha=0.3, color='0.9')
ax.set_xticks([])
ax.set_xlabel('Time')
ax.set_ylabel('Hitscore (Nr. of lit pixel)')
ax.semilogy()
plt.show()

# Experimental parameters

In [ ]:
# Load parameters from file
with h5py.File(filename_ol3, 'r') as f:
    photon_energy_ev = f['entry_1/FEL/photon_energy_eV_SLAC'][:]
    wavelength_nm    = f['entry_1/FEL/wavelength_nm_SLAC'][:]
    pulse_energy_mJ  = f['entry_1/detector_3/pulse_energy_mJ'][:]
print "Run %d has photon energy of (mean/std): %6.2f/%.3f eV" %(runnr, photon_energy_ev.mean(), photon_energy_ev.std())
print "Run %d has a wavelength  of (mean/std): %6.2f/%.3f nm" %(runnr, wavelength_nm.mean(),    wavelength_nm.std())
print "Run %d has pulse  energy of (mean/std): %6.2f/%.3f mJ" %(runnr, pulse_energy_mJ.mean(),  pulse_energy_mJ.std())

# Calculate initial support size

In [ ]:
# Parameters
pixelsize = 75 * 1e-6 # [m]
distance = 370 * 1e-3 # [m]
wavelength = wavelength_nm.mean() * 1e-9 # [m]
downsampling = 4.
shape = (256,256)

# Pixelsize in real space
dx = wavelength * distance / (shape[1] * pixelsize*downsampling) #[m/px]
print "One pixel in real space corresponds to %.1f nm" %(dx*1e9)
print "The support radius for PBCV (160 nm in diameter) should be: %d px" %((160. / (dx*1e9)) / 2.)

# Resolution

In [ ]:
x_to_q = lambda x: 2*np.sin(np.arctan2(x*pixelsize, distance)/2.) / wavelength
print "The resolution (at the edge)   is %.2f nm" %(1e9/x_to_q(512.))
print "The resolution (in the corner) is %.2f nm" %(1e9/x_to_q(512.*np.sqrt(2)))